# Uniview module for Hanny's Voorwerp 

Working to create a 3D scene from [this image.](https://www.nasa.gov/mission_pages/hubble/science/space-oddity.html)

This will include two parts

1. A particle model for Hanny's Voorwerp (see the rawdata directory)

2. A model for the AGN, adapted from my Blazar model from [here.](https://github.com/ageller/UniviewBlazar)

*Aaron Geller, 2019*

### Imports and function definitions

In [ ]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"

import sys, os, shutil, errno, string, urllib

sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
#print(sys.path)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [57]:
VoorwerpTemplate = """mesh
{  
    data blazar ./modules/$folderName/BlazarSolidModel_v2.2.obj
    data galaxy    ./modules/$folderName/galaxy.raw
    data voorwerp    ./modules/$folderName/voorwerp.raw

#    cullRadius $cr
    glslVersion 330
    enableTypeCheck true
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec1f cloudAlpha 0.2 | public | desc "clouds alpha" | widget slider | range 0 1
            vec1f jetAlpha 1.0 | public | desc "jet alpha" | widget slider | range 0 1
            vec1f jetLength 5.0 | public | desc "jet length" | widget slider | range 0 10
            vec1f galaxyAlpha 0.005 | public | desc "galaxy alpha" | widget slider | range 0 1
            vec1f galaxyPsize 10.0 | public | desc "galaxy particle size" | widget slider | range 0 20
            vec1f galaxyScale 50.0 | public | desc "galaxy size scale" | widget slider | range 0 100
            vec3f galaxyColor 1.0 1.0 1.0 | public | desc "galaxy color r,g,b"
            vec1f galaxyRotationX 1.32571 | public | desc "galaxy X rotation angle" | widget slider | range 0 6.283
            vec1f galaxyRotationY 0.06911 | public | desc "galaxy Y rotation angle" | widget slider | range 0 6.283
            vec1f galaxyRotationZ 0.408395 | public | desc "galaxy Z rotation angle" | widget slider | range 0 6.283
            
            vec1f voorwerpAlpha 0.1 | public | desc "voorwerp alpha" | widget slider | range 0 1
            vec1f voorwerpPsize 0.04 | public | desc "voorwerp particle size" | widget slider | range 0 20
            vec1f voorwerpScale 1.0 | public | desc "voorwerp size scale" | widget slider | range 0 100
            vec1f voorwerpOffsetX 190 | public | desc "voorwerp particle offset x" | widget slider | range -1000 1000
            vec1f voorwerpOffsetY 120 | public | desc "voorwerp particle offset y" | widget slider | range -1000 1000
            vec1f voorwerpOffsetZ -425 | public | desc "voorwerp particle offset z" | widget slider | range -1000 1000 
            vec1f voorwerpRotationX 0.0 | public | desc "voorwerp X rotation angle" | widget slider | range 0 6.283
            vec1f voorwerpRotationY 0.772809 | public | desc "voorwerp Y rotation angle" | widget slider | range 0 6.283
            vec1f voorwerpRotationZ 5.29029 | public | desc "voorwerp Z rotation angle" | widget slider | range 0 6.283
        }
    }

    ############# AGN
    pass 
    {
        
        useDataObject blazar
        
        shader
        {
            
            type defaultMeshShader
            {
                mtrl jetMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/jet.gs
                fragmentShader ./modules/$folderName/jet.fs
                
                stateManagerVar __objectName__.jetAlpha  jetAlpha
                stateManagerVar __objectName__.jetLength  jetLength
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
                
            }            
            type defaultMeshShader
            {
                mtrl blackHoleMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/blazar.fs
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ
            }

            type defaultMeshShader
            {
                mtrl accretionDiskMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/accretionDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }  
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ
            }

            type defaultMeshShader
            {
                mtrl exteriorDiskMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/exteriorDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ
            }

            type defaultMeshShader
            {
                mtrl cloudMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/blazar.gs #jet.gs
                fragmentShader ./modules/$folderName/clouds.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.cloudAlpha  cloudAlpha
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }


        }
        
            
    }
    
    ############# Spiral galaxy particles
    pass
    {
        useDataObject galaxy
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/galaxy.gs
                vertexShader ./modules/$folderName/voorwerp.vs
                fragmentShader   ./modules/$folderName/voorwerp.fs
                
                stateManagerVar __objectName__.galaxyPsize  userPsize
                stateManagerVar __objectName__.galaxyScale  userScale
                stateManagerVar __objectName__.galaxyAlpha  userAlpha
                stateManagerVar __objectName__.galaxyRotationX  userRotationX
                stateManagerVar __objectName__.galaxyRotationY  userRotationY
                stateManagerVar __objectName__.galaxyRotationZ  userRotationZ
                stateManagerVar __objectName__.galaxyColor  galaxyColor
                stateManagerVar __objectName__.voorwerpOffsetX voorwerpOffsetX
                stateManagerVar __objectName__.voorwerpOffsetY voorwerpOffsetY
                stateManagerVar __objectName__.voorwerpOffsetZ voorwerpOffsetZ
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }  
    }
    
    ############# Hanny's Voorwerp
    pass
    {
        useDataObject voorwerp
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/voorwerp.gs
                vertexShader ./modules/$folderName/voorwerp.vs
                fragmentShader   ./modules/$folderName/voorwerp.fs
                
                stateManagerVar __objectName__.voorwerpPsize  userPsize
                stateManagerVar __objectName__.voorwerpScale  userScale
                stateManagerVar __objectName__.voorwerpAlpha  userAlpha
                stateManagerVar __objectName__.voorwerpRotationX  userRotationX
                stateManagerVar __objectName__.voorwerpRotationY  userRotationY
                stateManagerVar __objectName__.voorwerpRotationZ  userRotationZ

                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    


    
}"""

### Voorwerp Class

In [58]:
class VoorwerpClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("voorwerp.raw",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("voorwerp.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("voorwerp.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("voorwerp.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("BlazarSolidModel_v2.2.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("galaxy.raw",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("blazar.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("accretionDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.vs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("clouds.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("galaxy.gs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(VoorwerpTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [59]:
model = VoorwerpClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [65]:
scene.setname("HannysVoorwerp")
scene.setparent("MilkyWay")
#scene.setunit(100000000000.0) #this seems like the correct size scale, but it is not holding it's position at a far zoom level
scene.setunit(1e6) #sets the size scale
scene.setentrydist(1e5) #not sure what this does
scene.setstaticposition(-148855900, 103197681, 125571970)
#scene.setstaticposition(-1488.55900, 1031.97681, 1255.71970)

model.object.setname("HannysVoorwerp")
model.object.setcoord(scene.name)
model.object.setcameraradius(1e5) #I think this defines some far clipping plane? but maybe not...
model.object.settargetradius(200000) #determines how far away the camera is when you jump to object
model.object.setguiname("/KavliLecture/Zooniverse/HannysVoorwerp")
model.object.setscalefactor(1e4) #not sure what this does
model.object.showatstartup(False)

#model.cr = 10000

modinfo.setname("HannysVoorwerp")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup> and Chris Lintott<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Oxford University")
modinfo.cleardependencies()
modinfo.setdesc("A 3D model of Hanny's Voorwerp")
modinfo.setversion("1.0")

generator.generate("HannysVoorwerp",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload;")


## To reload

In [13]:
from config import Settings

In [122]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload;")

### Create colormap texture

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('RdPu')

plt.show()

## Hanny's Voorwerp position 

*From [this arcticle.](https://academic.oup.com/mnras/article/399/1/129/1086654)*

In [ ]:
from astropy.coordinates import SkyCoord, Distance
from astropy import units

In [ ]:
RA = 145.2675*units.deg #right ascension
Dec= 34.7325*units.deg #declination
dist = 220.4*10**6 *units.pc

coord = SkyCoord(RA, Dec, dist)

print(coord.cartesian)

In [ ]:
import numpy as np
print(np.pi/2., np.pi)
